# S04_SparkSTFLocatel

curl -L https://datos.cdmx.gob.mx/dataset/529aac27-d1c1-426f-8c45-fd76fba43bf4/resource/44913088-806d-4f80-acca-1409a8225e9c/download/locatel0311-2024.csv | hdfs dfs -put -f - /tmp/amd/locatel0311/locatel0311-2024.csv
 
curl -L https://datos.cdmx.gob.mx/dataset/529aac27-d1c1-426f-8c45-fd76fba43bf4/resource/1ece1ddf-3e82-44f4-9435-486b1b9167f5/download/locatel0311-2023.csv | hdfs dfs -put -f - /tmp/amd/locatel0311/locatel0311-2023.csv
 
curl -L https://archivo.datos.cdmx.gob.mx/suac/locatel0311-2022.csv | hdfs dfs -put -f - /tmp/amd/locatel0311/locatel0311-2022.csv
 
curl -L https://archivo.datos.cdmx.gob.mx/suac/locatel0311-2021.csv | hdfs dfs -put -f - /tmp/amd/locatel0311/locatel0311-2021.csv
 
curl -L https://archivo.datos.cdmx.gob.mx/suac/locatel0311-2020.csv | hdfs dfs -put -f - /tmp/amd/locatel0311/locatel0311-2020.csv
 
curl -L https://archivo.datos.cdmx.gob.mx/suac/locatel0311-2019.csv | hdfs dfs -put -f - /tmp/amd/locatel0311/locatel0311-2019.csv


In [1]:
import pyspark
print(pyspark.__version__)

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("locatel.txt").getOrCreate()

3.5.3


24/10/19 15:36:40 WARN Utils: Your hostname, Shie-Hassaikai resolves to a loopback address: 127.0.1.1; using 192.168.100.17 instead (on interface wlp1s0)
24/10/19 15:36:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/19 15:36:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [22]:
from pyspark.sql.functions import col, sum, to_date, to_timestamp, regexp_replace,date_format
from pyspark.sql.types import IntegerType, DoubleType, FloatType

dir_hdfs="locatel2024.csv";

In [23]:
df = spark.read.csv(dir_hdfs, header=True, inferSchema=True)

In [24]:
df.printSchema()

print(f"Número de renglones: {df.count()}")
print(f"Número de columnas: {len(df.columns)}")
print(f"Columnas: {df.columns}")

root
 |-- id_folio: integer (nullable = true)
 |-- fecha_solicitud: date (nullable = true)
 |-- hora_solicitud: timestamp (nullable = true)
 |-- tipo_de_entrada: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- colonia_solicitud: string (nullable = true)
 |-- alcaldia_solicitud: string (nullable = true)
 |-- codigo_postal_solicitud: string (nullable = true)
 |-- estatus: string (nullable = true)
 |-- fecha_concluido: string (nullable = true)
 |-- alcaldia_catalogo: string (nullable = true)
 |-- colonia_catalogo: string (nullable = true)
 |-- longitud: string (nullable = true)
 |-- latitud: string (nullable = true)

Número de renglones: 113147
Número de columnas: 16
Columnas: ['id_folio', 'fecha_solicitud', 'hora_solicitud', 'tipo_de_entrada', 'tema_solicitud', 'sexo', 'edad', 'colonia_solicitud', 'alcaldia_solicitud', 'codigo_postal_solicitud', 'estatus', 'fecha_concluido', 'alcaldia_cata

In [25]:
df.show(truncate=False)

+--------+---------------+-------------------+--------------------------------+------------------------------------+---------------+----+-----------------+------------------+-----------------------+---------------------+---------------+-----------------+----------------+--------+---------------+
|id_folio|fecha_solicitud|hora_solicitud     |tipo_de_entrada                 |tema_solicitud                      |sexo           |edad|colonia_solicitud|alcaldia_solicitud|codigo_postal_solicitud|estatus              |fecha_concluido|alcaldia_catalogo|colonia_catalogo|longitud|latitud        |
+--------+---------------+-------------------+--------------------------------+------------------------------------+---------------+----+-----------------+------------------+-----------------------+---------------------+---------------+-----------------+----------------+--------+---------------+
|2399150 |2024-02-02     |2024-10-19 01:32:03|APP CDMX                        |DESAZOLVE                     

In [42]:
datos = df.select(['fecha_solicitud','hora_solicitud','tema_solicitud','sexo',
                   'edad','estatus','alcaldia_solicitud','colonia_catalogo','latitud','longitud'])

datos.printSchema()
datos.count()

root
 |-- fecha_solicitud: date (nullable = true)
 |-- hora_solicitud: timestamp (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- estatus: string (nullable = true)
 |-- alcaldia_solicitud: string (nullable = true)
 |-- colonia_catalogo: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



113147

In [29]:
datos.filter('sexo = "FEMENINO"').count()

51250

In [43]:
datos = df.select(['fecha_solicitud','hora_solicitud','tema_solicitud','sexo',
                   'edad','estatus','alcaldia_solicitud','colonia_catalogo','latitud','longitud']).filter('sexo = "FEMENINO"').orderBy('fecha_solicitud')
datos.show()

+---------------+-------------------+--------------------+--------+----+--------------------+-------------------+--------------------+------------------+----------------+
|fecha_solicitud|     hora_solicitud|      tema_solicitud|    sexo|edad|             estatus| alcaldia_solicitud|    colonia_catalogo|           latitud|        longitud|
+---------------+-------------------+--------------------+--------+----+--------------------+-------------------+--------------------+------------------+----------------+
|     2024-01-01|2024-10-19 11:29:26|                OTRO|FEMENINO|  NA|             CERRADO|         IZTAPALAPA|                  NA|                NA|              NA|
|     2024-01-01|2024-10-19 17:59:19|           ALUMBRADO|FEMENINO|  NA|             CERRADO|            TLALPAN|  Residencial Acoxpa|   19.297003237802|-99.125651430686|
|     2024-01-01|2024-10-19 11:04:11|                OTRO|FEMENINO|  NA|             CERRADO|         IZTAPALAPA|          Polvorilla|          1

In [44]:
if datos is not None:
    datos = datos.withColumn('fecha_solicitud', to_date(col('fecha_solicitud'),"yyyy-MM-dd")).withColumn('edad',col("fecha_solicitud").cast(FloatType())).\
        withColumn('latitud',col("latitud").cast(DoubleType())).withColumn('longitud',col("longitud").cast(DoubleType()))

datos.printSchema()

root
 |-- fecha_solicitud: date (nullable = true)
 |-- hora_solicitud: timestamp (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: float (nullable = true)
 |-- estatus: string (nullable = true)
 |-- alcaldia_solicitud: string (nullable = true)
 |-- colonia_catalogo: string (nullable = true)
 |-- latitud: double (nullable = true)
 |-- longitud: double (nullable = true)



In [45]:
datos.show()

+---------------+-------------------+--------------------+--------+----+--------------------+-------------------+--------------------+---------------+----------------+
|fecha_solicitud|     hora_solicitud|      tema_solicitud|    sexo|edad|             estatus| alcaldia_solicitud|    colonia_catalogo|        latitud|        longitud|
+---------------+-------------------+--------------------+--------+----+--------------------+-------------------+--------------------+---------------+----------------+
|     2024-01-01|2024-10-19 01:02:20|   ASESORÍA JURÍDICA|FEMENINO|NULL|             CERRADO|            TLALPAN|            La Palma|19.250095911703|-99.157668028529|
|     2024-01-01|2024-10-19 03:55:09|SOLICITUD DE VIGI...|FEMENINO|NULL|             CERRADO|          IZTACALCO|   Nueva Santa Anita| 19.39858556948|-99.128532174844|
|     2024-01-01|2024-10-19 09:56:19|                OTRO|FEMENINO|NULL|             CERRADO|         IZTAPALAPA|          Polvorilla|     19.3183916|     -99.0

## Clase 27/09/2024

In [48]:
#Contar nulos
datos.select([sum(col(c).isNull().cast(IntegerType())).alias(c) for c in datos.columns]).show()

+---------------+--------------+--------------+----+-----+-------+------------------+----------------+-------+--------+
|fecha_solicitud|hora_solicitud|tema_solicitud|sexo| edad|estatus|alcaldia_solicitud|colonia_catalogo|latitud|longitud|
+---------------+--------------+--------------+----+-----+-------+------------------+----------------+-------+--------+
|              0|             0|             0|   0|51250|      0|                 0|               0|  31913|   32767|
+---------------+--------------+--------------+----+-----+-------+------------------+----------------+-------+--------+



In [53]:
from pyspark.sql.functions import hour, minute, second, year, month, dayofmonth,weekofyear

datos_tiempo = datos.select(
    col("edad"),
    col("estatus"),
    col("tema_solicitud"),
    year(col("fecha_solicitud")).alias("anio"),
    month(col("fecha_solicitud")).alias("mes"),
    dayofmonth(col("fecha_solicitud")).alias("dia"),
    hour(col("hora_solicitud")).alias("hora"),
    minute(col("hora_solicitud")).alias("minuto"),
    second(col("hora_solicitud")).alias("segundo"),
    weekofyear(col("fecha_solicitud")).alias("semana")  
)

datos_tiempo.printSchema()
datos_tiempo.describe().show(truncate=False)

root
 |-- edad: float (nullable = true)
 |-- estatus: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- anio: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- dia: integer (nullable = true)
 |-- hora: integer (nullable = true)
 |-- minuto: integer (nullable = true)
 |-- segundo: integer (nullable = true)
 |-- semana: integer (nullable = true)



+-------+----+--------+---------------------------+------+-------------------+-----------------+------------------+------------------+------------------+------------------+
|summary|edad|estatus |tema_solicitud             |anio  |mes                |dia              |hora              |minuto            |segundo           |semana            |
+-------+----+--------+---------------------------+------+-------------------+-----------------+------------------+------------------+------------------+------------------+
|count  |0   |51250   |51250                      |51250 |51250              |51250            |51250             |51250             |51250             |51250             |
|mean   |NULL|NULL    |NULL                       |2024.0|1.4848780487804878 |16.51488780487805|12.822068292682927|29.612058536585366|29.566985365853657|5.047551219512195 |
|stddev |NULL|NULL    |NULL                       |0.0   |0.49977615016451876|8.528361720190427|3.2661728439798807|17.24426313225204 |1

In [64]:
columnas = ['anio','mes','dia','hora','minuto','segundo','semana']
datos_tiempo.select(columnas).describe().show()
datos_tiempo.select(columnas).describe()

+-------+------+-------------------+-----------------+------------------+------------------+------------------+------------------+
|summary|  anio|                mes|              dia|              hora|            minuto|           segundo|            semana|
+-------+------+-------------------+-----------------+------------------+------------------+------------------+------------------+
|  count| 51250|              51250|            51250|             51250|             51250|             51250|             51250|
|   mean|2024.0| 1.4848780487804878|16.51488780487805|12.822068292682927|29.612058536585366|29.566985365853657| 5.047551219512195|
| stddev|   0.0|0.49977615016451876|8.528361720190427|3.2661728439798807| 17.24426313225204|17.298404705267156|2.4362187928499424|
|    min|  2024|                  1|                1|                 0|                 0|                 0|                 1|
|    max|  2024|                  2|               31|                23|          

DataFrame[summary: string, anio: string, mes: string, dia: string, hora: string, minuto: string, segundo: string, semana: string]

In [65]:
#Ver datos únicos
datos_tiempo.select("tema_solicitud").distinct().count()

47

In [66]:
datos_tiempo.select("tema_solicitud").distinct().show(truncate=False)

+---------------------------------+
|tema_solicitud                   |
+---------------------------------+
|VERIFICACIÓN ADMINISTRATIVA      |
|BACHEO                           |
|REPARACION DE EMPEDRADO          |
|BECAS                            |
|FALTA DE AGUA                    |
|REINSCRIPCION AL CENDI           |
|LLAVE CDMX                       |
|ALARMAS VECINALES                |
|SOLICITUD/SEGURO DE DESEMPLEO    |
|SOLICITUD DE VIGILANCIA          |
|PODA / RETIRO ARBOL              |
|TRAMITES / INFO VEHICULAR        |
|ALERTA SISMICA                   |
|SOLICITUD DE CONCERTACIÓN VECINAL|
|DESAZOLVE                        |
|QUEJA FUNCIONARIO                |
|ASISTENCIA VETERINARIA           |
|SOLICITUD ESTUDIO SOCIOECONOMICO |
|MANTENIMIENTO PARQUE / AREA VERDE|
|BARBECHO / CHAPONEO              |
+---------------------------------+
only showing top 20 rows



In [69]:
datos_tiempo.groupBy("tema_solicitud").count().orderBy("count", ascending=False).show(100, truncate=False)

+----------------------------------------+-----+
|tema_solicitud                          |count|
+----------------------------------------+-----+
|OTRO                                    |32974|
|FALTA DE AGUA                           |3127 |
|ALUMBRADO                               |2463 |
|PODA / RETIRO ARBOL                     |2290 |
|FUGA DE AGUA                            |1073 |
|DESAZOLVE                               |995  |
|MANTENIMIENTO VÍA PÚBLICA               |915  |
|QUEJA DE TRANSPORTE PUBLICO             |751  |
|BACHEO                                  |708  |
|VERIFICACIÓN ADMINISTRATIVA             |606  |
|RETIRO CASCAJO, ESCOMBRO, AZOLVE, RAMAS |593  |
|BARBECHO / CHAPONEO                     |591  |
|VEHÍCULO ABANDONADO / CHATARRIZACIÓN    |480  |
|ASISTENCIA SOCIAL                       |297  |
|RECOLECCIÓN BASURA                      |297  |
|QUEJA FUNCIONARIO                       |295  |
|SOLICITUD DE VIGILANCIA                 |279  |
|MANTENIMIENTO DE CO

In [71]:
resultado = datos_tiempo.groupBy("tema_solicitud").count().orderBy("count", ascending=False)

# Muestra los resultados
resultado.show(100, truncate=False)

+----------------------------------------+-----+
|tema_solicitud                          |count|
+----------------------------------------+-----+
|OTRO                                    |32974|
|FALTA DE AGUA                           |3127 |
|ALUMBRADO                               |2463 |
|PODA / RETIRO ARBOL                     |2290 |
|FUGA DE AGUA                            |1073 |
|DESAZOLVE                               |995  |
|MANTENIMIENTO VÍA PÚBLICA               |915  |
|QUEJA DE TRANSPORTE PUBLICO             |751  |
|BACHEO                                  |708  |
|VERIFICACIÓN ADMINISTRATIVA             |606  |
|RETIRO CASCAJO, ESCOMBRO, AZOLVE, RAMAS |593  |
|BARBECHO / CHAPONEO                     |591  |
|VEHÍCULO ABANDONADO / CHATARRIZACIÓN    |480  |
|ASISTENCIA SOCIAL                       |297  |
|RECOLECCIÓN BASURA                      |297  |
|QUEJA FUNCIONARIO                       |295  |
|SOLICITUD DE VIGILANCIA                 |279  |
|MANTENIMIENTO DE CO

## Clase 30/09/2024

In [72]:
#Localhost:4040
datos_tiempo.printSchema()

root
 |-- edad: float (nullable = true)
 |-- estatus: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- anio: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- dia: integer (nullable = true)
 |-- hora: integer (nullable = true)
 |-- minuto: integer (nullable = true)
 |-- segundo: integer (nullable = true)
 |-- semana: integer (nullable = true)



In [74]:
datos_tiempo.filter(datos_tiempo.anio==2024).count()

51250

In [76]:
datos_tiempo.filter(datos_tiempo.anio==2024).show(truncate=False)

+----+---------------------+---------------------------+----+---+---+----+------+-------+------+
|edad|estatus              |tema_solicitud             |anio|mes|dia|hora|minuto|segundo|semana|
+----+---------------------+---------------------------+----+---+---+----+------+-------+------+
|NULL|CERRADO              |ASESORÍA JURÍDICA          |2024|1  |1  |1   |2     |20     |1     |
|NULL|CERRADO              |SOLICITUD DE VIGILANCIA    |2024|1  |1  |3   |55    |9      |1     |
|NULL|CERRADO              |OTRO                       |2024|1  |1  |9   |56    |19     |1     |
|NULL|AUTORIDAD RESPONSABLE|BACHEO                     |2024|1  |1  |9   |8     |34     |1     |
|NULL|CERRADO              |SOLICITUD DE VIGILANCIA    |2024|1  |1  |10  |23    |57     |1     |
|NULL|CERRADO              |FALTA DE AGUA              |2024|1  |1  |10  |28    |16     |1     |
|NULL|CERRADO              |SOLICITUD DE VIGILANCIA    |2024|1  |1  |9   |31    |37     |1     |
|NULL|CERRADO              |OT

In [84]:
datos.printSchema()

root
 |-- fecha_solicitud: date (nullable = true)
 |-- hora_solicitud: timestamp (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: float (nullable = true)
 |-- estatus: string (nullable = true)
 |-- alcaldia_solicitud: string (nullable = true)
 |-- colonia_catalogo: string (nullable = true)
 |-- latitud: double (nullable = true)
 |-- longitud: double (nullable = true)



In [83]:
datos_tiempo.printSchema()

root
 |-- edad: float (nullable = true)
 |-- estatus: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- anio: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- dia: integer (nullable = true)
 |-- hora: integer (nullable = true)
 |-- minuto: integer (nullable = true)
 |-- segundo: integer (nullable = true)
 |-- semana: integer (nullable = true)



In [90]:
datos_tiempo.select(['edad','anio','dia','hora','estatus']).\
filter((datos_tiempo.anio==2024) & (datos_tiempo.anio <=6)).\
filter(datos_tiempo.dia=="Sunday").\
filter(datos_tiempo.edad>0).\
show()

+----+----+---+----+-------+
|edad|anio|dia|hora|estatus|
+----+----+---+----+-------+
+----+----+---+----+-------+



In [92]:
datos.groupBy("alcaldia_solicitud").count().show()

+--------------------+-----+
|  alcaldia_solicitud|count|
+--------------------+-----+
|          XOCHIMILCO|  340|
|LA MAGDALENA CONT...|   62|
|        AZCAPOTZALCO|  829|
|            COYOACAN|  689|
|             TLALPAN|  558|
|                  NA|13906|
|             TLAHUAC|  175|
|CUAJIMALPA DE MOR...|   56|
|       BENITO JUAREZ|  542|
|          MILPA ALTA|  441|
|   GUSTAVO A. MADERO| 1213|
|          IZTAPALAPA|29427|
|           IZTACALCO|  969|
|          CUAUHTEMOC| 1152|
| VENUSTIANO CARRANZA|  191|
|      MIGUEL HIDALGO|  233|
|      ALVARO OBREGON|  467|
+--------------------+-----+



In [93]:
datos.groupBy("alcaldia_solicitud").count().show(100,truncate=False)

+----------------------+-----+
|alcaldia_solicitud    |count|
+----------------------+-----+
|XOCHIMILCO            |340  |
|LA MAGDALENA CONTRERAS|62   |
|AZCAPOTZALCO          |829  |
|COYOACAN              |689  |
|TLALPAN               |558  |
|NA                    |13906|
|TLAHUAC               |175  |
|CUAJIMALPA DE MORELOS |56   |
|BENITO JUAREZ         |542  |
|MILPA ALTA            |441  |
|GUSTAVO A. MADERO     |1213 |
|IZTAPALAPA            |29427|
|IZTACALCO             |969  |
|CUAUHTEMOC            |1152 |
|VENUSTIANO CARRANZA   |191  |
|MIGUEL HIDALGO        |233  |
|ALVARO OBREGON        |467  |
+----------------------+-----+



In [95]:
datos.groupBy("alcaldia_solicitud").agg({"edad":"avg"}).show(100, truncate=False)

+----------------------+---------+
|alcaldia_solicitud    |avg(edad)|
+----------------------+---------+
|XOCHIMILCO            |NULL     |
|LA MAGDALENA CONTRERAS|NULL     |
|COYOACAN              |NULL     |
|AZCAPOTZALCO          |NULL     |
|TLALPAN               |NULL     |
|NA                    |NULL     |
|TLAHUAC               |NULL     |
|CUAJIMALPA DE MORELOS |NULL     |
|BENITO JUAREZ         |NULL     |
|MILPA ALTA            |NULL     |
|GUSTAVO A. MADERO     |NULL     |
|IZTAPALAPA            |NULL     |
|IZTACALCO             |NULL     |
|CUAUHTEMOC            |NULL     |
|VENUSTIANO CARRANZA   |NULL     |
|MIGUEL HIDALGO        |NULL     |
|ALVARO OBREGON        |NULL     |
+----------------------+---------+



In [97]:
datos.write.csv("locateltiempo.cvs")

In [98]:
type(datos)

pyspark.sql.dataframe.DataFrame

# S05_SparkSTFLocatel

In [ ]:
dft.write.mode("").cvs("hdfs://namenode:9000/tmp/amd/locateltiempo")


In [ ]:
type(dft)

In [ ]:
dft.select("*", (dft.edad * 12).alias("edad_meses")).show(100, truncate = False)

In [ ]:
dft.select("*", (dft.edad * 12).\
           alias("edad_meses")write.mode("overwrite").\
            cvs("hdfs://namenode:9000/tmp/amd/locateltiempo", header = True))

In [ ]:
dft.select("*", (dft.edad * 12).\
           alias("edad_meses")write.mode("overwrite").\
            cvs("hdfs://namenode:9000/tmp/amd/locateltiempo", header = True))

In [ ]:
from pyspark.sql.functions import count
datos_tiempo.groupBy("anio", "mes").agg(count("*").alias("nr")).show(truncate = False)

In [ ]:
datos_tiempo.select(["edad","estatus","tema_solicitud", "anio","mes"])\
            .partitionBy("anio")\
            .write.mode("append")\
            .cvs("hdfs://namenode:9000/tmp/amd/locateltiempobyanio", header = True)

In [ ]:
dfp = spark.read.option(header = True)\
           .cvs("hdfs://namenode:9000/tmp/amd/locateltiempobyanioymes")

In [ ]:
dfp.printSchema()

In [ ]:
dfp.show(30, truncate = False)

In [ ]:
dfp.filter('mes = 7 AND anio = 2022').show()

## Clase 07/10/2024

In [ ]:
from pyspark.sql.functions import col, sum, to_date, to_timestamp, regexp_replace
from pyspark.sql.functions import date_format
from pyspark.sql.types import IntergerType, DoubleType, FloatType

In [ ]:
dir_hdfs = "hdfs://namenode:9000/tmp/amd/locatel0311"

In [ ]:
from pyspark.sql.functions import agg, max, min, count

dfg = datos.groupBy("alcaldia").\
    agg(avg("edad").alias("edad_promedio")
                max("edad").alias("edad_maxima")
                min("edad").alias("edad_minima")
                count("*").alias("no_registros"))

In [ ]:
dfg.groupBy("alcaldia_solicitud",ascendig = False).show(100, truncate=False)

In [ ]:
from pyspark.sql.functions import when

datos = datos.\
    withColumn("alcaldia_solicitud", when(datos.alcaldia_solicitud == "IZTACALCO", "iZTACALCO")).\
    otherwise(datos.alcaldia_solicitud)

In [ ]:
from os import truncate
datos.groupBy("alcaldia").\
    agg(avg("edad").alias("edad_promedio")
                max("edad").alias("edad_maxima")
                min("edad").alias("edad_minima")
                count("*").alias("no_registros")).\
    orderBy("alcaldia_solicitud",ascending = False).\
    show(100,truncate = False)

In [ ]:
sepomex_df = spark.read.\
    option("header", "True").option("delimiter", "|").\
    option("skipRows", "1")sch(schema).\
    csv("hdfs://namenode:9000/tmp/amd/sepomex")

In [ ]:
dfcp = sepomex_df.select(['d_codigo','D_mnpio','d_estado'])

In [ ]:
sepomex_df.printSchema()

In [ ]:
dfcp.count()

In [ ]:
dfcp.show(truncate = False)

In [ ]:
from pyspark.sql.functions import col, coalesce

datos_mapeados = datos.join(dfcp.select(["d_codigo", "D_mnpio"]), datos["codigo_postal_solicitud"] == dfcp["d_codigo"], "left")

In [ ]:
datos_mapeados.printSchema()

In [ ]:
datos_mapeados.select(["codigo_postal_solicitud", "alcaldia_solicitud", "d_codigo", "D_mnpio"]).show(100, truncate = False)

In [ ]:
datos_mapeados.withColumn("alcaldia", coalesce(col("D_mnpio"), col("alcaldia")))

In [ ]:
datos_mapeados.select(["edad", "alcadia", "codigo_postal_solicitud","d_codigo"]).show(100, truncate = False)

In [ ]:
#Verificar duplicados
dfcp_dup = dfcp.groupBy("d_codigo").agg(count("*").alias("count")).\
    filter("count" > 1)

print(dfcp_dup.count())
dfcp_dup.orderBy("count", ascending = True).show(100,truncate = False)

In [ ]:
#ver duplicados
dfcp.filter("d_codigo = 85203").show(truncate = False)

In [ ]:
#Eliminar duplicados
dfcp = dfcp.dropDuplicates("d_codigo")

In [ ]:
#Validar que ya no hay duplicados
dfcp.groupBy("d_codigo").agg(count("*").alias("count")).filter("count > 1").show()

In [ ]:
from pyspark.sql.functions import col, lpad, regexp_replace, trim
datos = datos.withColumn(
    "codigo_postal_solicitud", lpad (
        trim(
          regexp_replace (
              col(
                  "codigo_postal_solicitud"),"[0-9]", "")
        ), 5, "0")
)

In [ ]:
datos.select("codigo_postal_solicitud").show()

In [ ]:
datos.select("codigo_postal_solicitud" == "00000").\
    dropDup(["codigo_postal_solicitud"]).\
    filter("codigo_postal_solicitud") and (len(col("alcaldia")==5)).show()

In [ ]:
#Actualizar codigo postal, cambiarlo por alcaldía
datos = datos.withColumn("codigo_postal_solicitud" when (
         (col("codigo_postal_solicitud") = "00000")
            and (lenght(col("alcaldia")) = 5),
         col("alcaldia_solicitud")
    )
)

In [ ]:
datos.select("alcaldia","codigo_psotal_solicitud").filter(
    col("codigo_postal_solicitud") == col ("alcaldia")
).show()

In [ ]:
dfg = datos_mapeados.groupBy("alcaldia").agg(
    avg("edad").alias("edad_promedio"),
    max("edad").alias("edad_maxima"),
    min("edad").alias("edad_minima"),
    count("*").alias("no_registros")
    )

In [ ]:
dfg.count()

In [ ]:
dfg.orderBy("alcaldia_solicitud",ascendig = False).show(100,truncate = False)

In [ ]:
datos_mapeados.write.mode("overwrite").option("heeader","true").csv("hdfs://namenode:9000/tmp/amd/locatellimpio")